# Urban POMDP tutorial

This tutorial explains how to use the Urban POMDP model as a reinforcement learning environment. 
The source code can be found in [https://github.com/sisl/AutomotivePOMDPs/tree/master/src/generative_pomdps/urban]().

In [1]:
using Random
using POMDPs
using POMDPPolicies
using POMDPSimulators
using BeliefUpdaters
using AutomotiveDrivingModels
using AutomotiveSensors
using AutomotivePOMDPs
using AutoViz
using Reel

┌ Info: Recompiling stale cache file C:\Users\Maxime\.julia\compiled\v1.0\POMDPSimulators\i1HOp.ji for POMDPSimulators [e0d0a172-29c6-5d4e-96d0-f262df5d01fd]
└ @ Base loading.jl:1184
┌ Warning: Replacing module `Distributions`
└ @ Base loading.jl:872
┌ Info: Recompiling stale cache file C:\Users\Maxime\.julia\compiled\v1.0\AutomotiveSensors\NEFS3.ji for AutomotiveSensors [9ec66b5e-c605-11e8-1454-ede748df6f96]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file C:\Users\Maxime\.julia\compiled\v1.0\AutomotivePOMDPs\L1lsd.ji for AutomotivePOMDPs [dca8afc0-c686-11e8-0934-712707da73aa]
└ @ Base loading.jl:1184
┌ Warning: Module POMDPs with build ID 270205300908919 is missing from the cache.
│ This may mean POMDPs [a93abf59-7444-517b-a68a-c42f96afdd7d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Recompiling stale cache file C:\Users\Maxime\.julia\compiled\v1.0\RLInterface\BrdPi.ji for RLInterface [3065fe53-3878-5aed-aae7-ab

Define a random number generator to have reproducible simulations

In [2]:
rng = MersenneTwister(1);

## Initializing the environment

The scenario geometry and road topology is defined using the `UrbanParams` and `UrbanEnv` types. To check the fields of these objects use the command `fieldnames`. 

The urban environment is alway a T shape intersections with crosswalk. 

Here we define a single lane intersection with three crosswalks and set the stop line position.

In [3]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

Initialize the pomdp object. The pomdp is implemented according to the [POMDPs.jl generative interface](http://juliapomdp.github.io/POMDPs.jl/latest/generative.html).

It has various parameters:
- sensor: defines the type of sensor that you want to use, for sensor models see [AutomotiveSensors.jl](https://github.com/sisl/AutomotiveSensors.jl)
- ego_goal: the lane that the ego vehicle must reach
- max_cars: the maximum number of cars supported by the simulations (will define the dimension of the observation vector)
- max_peds: the maximum number of pedestrian
- max_obstacles: the maximum number of obstacles on the side of the road. Maximum 3 obstacles are supported. They are randomly generated according to an obstacle distribution (see pomdp.obs_dist)
- ego_start: the ego vehicle starting position as a longitudinal distance in the initial lane. 
- $\Delta$T: the decision frequency

In [13]:
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                   max_cars=5, 
                   max_peds=5, 
                   car_birth=0.5, 
                   ped_birth=0.2, 
                   max_obstacles=0, # no fixed obstacles
                   ego_start=20,
                   ΔT=0.1);

## Running simulation

One can run simulations using the [POMDPSimulators.jl](https://github.com/JuliaPOMDP/POMDPSimulators.jl) package. See [this notebook](https://github.com/JuliaPOMDP/POMDPExamples.jl/blob/master/notebooks/Running-Simulations.ipynb) for how to use it. Here we run a simple simulation with a `HistoryRecorder` and a single action policy. 


In [17]:
still_policy = FunctionPolicy(s -> UrbanAction(0.)) # no acceleration
up = NothingUpdater() # no belief update, to use the observation as input to the policy see PreviousObservationUpdater from BeliefUpdaters.jl
s0 = initialstate(pomdp, rng) # generate an initial state
# set up the simulation
hr = HistoryRecorder(rng=rng, max_steps=400)
@time hist = simulate(hr, pomdp, still_policy, up, nothing, s0);

  0.496681 seconds (761.40 k allocations: 43.922 MiB, 2.66% gc time)


## Visualize simulation

To visualize a simulation we can generate a video using Reel.jl and the states stored in the history recorder.

In [11]:
state_history = state_hist(hist)
duration, fps, render_rec = animate_scenes(state_history, env)
speed_factor = 2
film = roll(render_rec, fps = speed_factor*fps, duration = duration/speed_factor)

Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_4D86.tmp", 0x00000000000000c9, 20.0, nothing)

## State and Observations

The state is represented by a `Scene` from [AutomotiveDrivingModels.jl](https://github.com/sisl/AutomotiveDrivingModels.jl). It is a datastructure containing the physical state of all the traffic participants (global frame position, frenet frame position, velocity, shape of the vehicle, id). 

The observation is represented by a `Vector{Float64}`. It is of dimension: $n_{features}\times(1 +max\_cars + max\_peds + max\_obstacles)$

$n_{features} = 4$ 

- dims 1 to 4 are the ego vehicle state as ego_pos_x, ego_pos_y, theta, v 
- then comes each car state as pos_x - ego_pos_x, pos_y - ego_pos_y, theta, v
- then pedestrians states: pos_x - ego_pos_x, pos_y - ego_pos_y, theta, v
- then obstacles states: widht, height, center_x - ego_pos_x, center_y - ego_pos_y

Each quantity is normalized to be between 0 and 1.

## Interact with the environment

To interact with the environment one can use the function from the POMDPs.jl generative interface (see [doc](http://juliapomdp.github.io/POMDPs.jl/latest/generative.html))

Another alternative is to use the [RLInterface.jl](https://github.com/JuliaPOMDP/RLInterface.jl) package that provide the ability to interface with existing python solvers. 